In [1]:
import pandas as pd
import numpy as np
import math
from collections import defaultdict
import queue
import heapq

In [2]:

#class for the environment of the n^2 -1 puzzle
#this is also going to have the function for the parity check
class puzzle : 
    def __init__(self):
        n = int(input("Enter the dimension of the matrix : "))
        self.dimension = n
        self.mat = np.zeros((self.dimension , self.dimension))
        self.red_x = -1
        self.red_y = -1
    #this is to take the matrix as the input 
    #in order to take the blank we take * as the input
    def enter_matrix(self):
        print("Enter the elements of the matrix : ")
        for i in range(self.dimension):
            temp  = input().strip().split()
            for j , x in zip(range(self.dimension) , temp):
                if (int(x) == 2):                    #symbol for the red block
                    self.mat[i , j] = 2
                    self.red_x = i
                    self.red_y = j
                else:
                    self.mat[i , j] = int(x)
    
    
    def duplicate(self , matrix):
            self.mat = np.copy(matrix)
            for i in range(self.dimension):
                for j in range(self.dimension):
                    if (self.mat[i , j] == 2):
                        self.red_x = i
                        self.red_y = j
                        
                        
    def isValid (self, x , y):
        if (x >= 0 and x < self.dimension  and y >=0 and y < self.dimension):
            return True
        return False
    
    
    #1 for left
    #2 for right
    #3 for up
    #4 for down
    
    #0 for blocked block
    #2 for red block
    #1 for empty block
    def get_new (self, x , y , action):
        movements = [(0, -1), (0, 1) , (-1 , 0) , (1 , 0)]
        move_x , move_y = movements [action - 1]
        return (x + move_x , y+move_y)
    
    
    def movement_coordinates( self , x , y , action , reverse):
        
        if (self.mat[x,y] == 0):           #this is the block
            return False
        if (reverse == False):
            movements = [(0, -1), (0, 1) , (-1 , 0) , (1 , 0)]        
        else:
            movements = [(0 , 1) , (0 , -1 ) , (1 , 0) , (-1 , 0)]
        move_x , move_y = movements [action - 1]
        
        new_x = move_x + x
        new_y = move_y + y
        
        if (not self.isValid(new_x , new_y)):
            return False
        
        
        #if the current block is the red space
        if (self.mat[x , y] == 2):
            if (self.mat[new_x , new_y] == 1):     #if the new block is empty
                self.mat[new_x][new_y] = 2
                self.mat[x][y] = 1
                self.red_x = new_x
                self.red_y = new_y
                return True
            else:
                return False
        
        #if the current block is the white space
        if (self.mat[x , y] == 1):
            if (self.mat[new_x][new_y] == 2):          #we are moving the red block as well
                self.mat[new_x][new_y] = 1
                self.mat[x , y] = 2
                self.red_x = x
                self.red_y = y
                return True
            
            elif (self.mat[new_x][new_y] == 0):
                self.mat[new_x][new_y] = 1
                self.mat[x][y] = 0
                return True
            else:
                return False

    def goal_satisfied (self):
        if (self.mat[0 , self.dimension - 1] == 2):
            return True
        return False
            


    
    #this is to return the manhattan heuristic
    def h_manhattan (self):
        return (self.red_x + self.red_y)
                    

In [3]:
env = puzzle()
env.enter_matrix()
print("The matrix is : ")
print(env.mat)
start = np.copy(env.mat)

Enter the dimension of the matrix : 3
Enter the elements of the matrix : 
1 0 0
0 1 0
0 0 2
The matrix is : 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 2.]]


In [4]:
print(env.goal_satisfied())

False


In [5]:
print(env.red_x , env.red_y)

2 2


In [6]:
print(env.dimension)

3


In [7]:
env.movement_coordinates(2 , 2 , 3 , True)
print(env.mat)
print(env.red_x , env.red_y)

[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 2.]]
2 2


In [8]:
#solving using the A* heuristic
def heuristics():
    env.duplicate(start)
    current_string = np.array_str(env.mat)   #this is the matrix to convert the matrix in the string form to keep 
                                             #track of the explored states
    
    d = dict()                #this is to store the node
                              #first element is the matrix which is the parent of the string
                              #second is the total cost to go till now
                              #third is the heuristic
                    
    q = []                    #this is the list and in some sense we are going to treat it as the priority queue
    heapq.heapify (q)
    
    explored = []         #this is to maintain the set of the explored states
                             #this is to initialised to empty
        
    frontier = []            #frontier is the list that is used to keep track of the elements that are there in the queue
    
    
    d[current_string] = ("-1" , 0 , env.h_manhattan())         #this is to put the current string in the state and initialise all it parameters
    ans = False              #this is to see if the solution is found or not
        
    counter = 0              #to see how many steps have been taken to solve the problem
    counter_prior = 0
    heapq.heappush(q , (d[current_string][1] + d[current_string][2] , counter_prior ,np.copy(env.mat) ) )   #this is to put the value of the heuristic in the queue
    frontier.append (current_string)
    
    while len(q) > 0 and ans == False:
        
    
        #print(frontier)
        #this is to print the number of steps
        counter += 1             #to count the number of steps taken
        #print("Number of steps : " , counter)
        
        
        #the element that is popped out from the heap have the fist element as the total estimated cost 
        #and the third element as the state from which it has beee taken
        temp = heapq.heappop (q)      #popping the element out of the queue
        
        matrix = temp[2]                          #taking the matrix out of it
        matrix_string = np.array_str (matrix)     #converting the matrix into the string
        frontier.remove(matrix_string)
        env.duplicate(matrix)                     #creating the duplicate of the environment
        #print (matrix)
        
        
        if (env.goal_satisfied() == True):  #to check if this is the goal state if yes break out of the loop
            print(env.red_x , env.red_y)
            ans = True
            break

        explored.append (matrix_string)              #to say that this node is explored
        
        dist = d[matrix_string][1] + d[matrix_string][2]      #getting the actual value out of the d
        if (temp[0] > dist):                      #if the value in the queue is greater than the current distance then discarding it
            continue                              #note this is for the lazy detection
        
        #this is to carry out each action
        #this is to explore the current string
        #moving just the first blank
        
        for x in range(env.dimension):
            for y in range(env.dimension):
                for action in [1 ,2 ,3 , 4]:
                    #print(x , y ,action)
                    new_x , new_y = env.get_new(x , y , action)
                    if (env.movement_coordinates(x , y , action , False)):
                        current_string = np.array_str (env.mat)
                        
                        #print(current_string)
                        if (current_string not in explored and current_string not in frontier):
                            d[current_string] = (matrix_string , d[matrix_string][1] + 1 , env.h_manhattan())
                            counter_prior += 1
                            heapq.heappush(q , (d[current_string][1] + d[current_string][2], counter_prior, np.copy(env.mat) ))
                            frontier.append(current_string)
                        
                        elif (current_string in frontier and d[current_string][1] > d[matrix_string][1] + 1):
                            d[current_string] = (matrix_string , "left b0" , d[matrix_string][2] + 1 , env.h_manhattan())
                            counter_prior += 1
                            heapq.heappush(q , (d[current_string][1] + d[current_string][2], counter_prior, np.copy(env.mat) ))
                            frontier.append(current_string)
                        env.movement_coordinates(new_x , new_y , action , True)
        
    if (ans == False):
        print("No Solution")
    else:
        
        current_string = np.array_str(env.mat)
        path = []
        length = 0
        while not d[current_string][0] == "-1":
            length += 1
            path.insert(0 , d[current_string][0])
            current_string = d[current_string][0]
        
        print("Total length  :" , length)
        for x in path:
            print(x)
        print(env.mat)
        


In [9]:
heuristics()

TypeError: get_new() takes 3 positional arguments but 4 were given